In [21]:
import hopsworks
from hsfs.client.exceptions import RestAPIError

# Establish connection to Hopsworks
project = hopsworks.login(
    api_key_value="dYTVrTVvbj6Qw82i.YGKHdS9snQYFgOADJIvLdvZ2n2S5BxIAOtvPUEmAyd56bvaG6xhhGyNM3nYbexaP"
)

# Access the Feature Store
fs = project.get_feature_store()

# List of feature groups and their versions
feature_groups = [
    {"name": "city_weather", "version": 1},
    {"name": "drivers_table", "version": 1},
    {"name": "routes_table", "version": 1},
    {"name": "routes_weather", "version": 1},
    {"name": "traffic_table", "version": 1},  # Corrected typo
    {"name": "truck_schedule_table", "version": 1},
    {"name": "trucks_table", "version": 1},
]
# Initialize an empty dictionary to hold DataFrames for each feature group
feature_group_data = {}

# Loop over the list of feature groups and download them
for fg in feature_groups:
    try:
        # Retrieve the feature group by its name and version
        feature_group = fs.get_feature_group(fg['name'], version=fg['version'])
        df = feature_group.read()  # Read the feature group as a DataFrame
        
        # Save the DataFrame in the dictionary with the feature group's name as the key
        feature_group_data[fg['name']] = df

        print(f"Downloaded feature group: {fg['name']} (version {fg['version']})")
        print(df.head())  # Optionally display the first few rows of the DataFrame

    except RestAPIError as e:
        print(f"Error downloading feature group: {fg['name']} (version {fg['version']})")
        print(e)

# After the loop, you will have each feature group stored in the `feature_group_data` dictionary
# Access each DataFrame like this: feature_group_data['city_weather'], etc.



Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1044630
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.63s) 
Downloaded feature group: city_weather (version 1)
      city_id  temp  wind_speed           description  precip  humidity  \
0  C-731988ba    52           1  Patchy rain possible     0.0        80   
1  C-841ebdcb    66           6         Partly cloudy     0.0        84   
2  C-ef47bdcd    10           6                 Sunny     0.0        57   
3  C-ff8c0c3c    34           8              Overcast     0.0       100   
4  C-3dbd8b2e    41          11         Partly cloudy     0.0        73   

   visibility  pressure     id                       event_time  \
0           6      1015   8790 2024-10-10 20:13:27.088206+00:00   
1           6      1020  52029 2024-10-10 20:13:27.08

In [22]:
feature_group_data.keys()

dict_keys(['city_weather', 'drivers_table', 'routes_table', 'routes_weather', 'traffic_table', 'truck_schedule_table', 'trucks_table'])

In [23]:
city_weather = feature_group_data['city_weather']
drivers_table = feature_group_data['drivers_table']
routes_table = feature_group_data['routes_table']
routes_weather = feature_group_data['routes_weather']
traffic_table = feature_group_data['traffic_table'] 
truck_schedule_table = feature_group_data['truck_schedule_table']
trucks_table = feature_group_data['trucks_table']


city_weather.head()

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,id,event_time,datetime
0,C-731988ba,52,1,Patchy rain possible,0.0,80,6,1015,8790,2024-10-10 20:13:27.088206+00:00,2019-02-14 05:00:00+00:00
1,C-841ebdcb,66,6,Partly cloudy,0.0,84,6,1020,52029,2024-10-10 20:13:27.088206+00:00,2019-01-07 20:00:00+00:00
2,C-ef47bdcd,10,6,Sunny,0.0,57,6,1036,13063,2024-10-10 20:13:27.088206+00:00,2019-02-08 06:00:00+00:00
3,C-ff8c0c3c,34,8,Overcast,0.0,100,6,1024,16910,2024-10-10 20:13:27.088206+00:00,2019-01-15 13:00:00+00:00
4,C-3dbd8b2e,41,11,Partly cloudy,0.0,73,6,1021,18928,2024-10-10 20:13:27.088206+00:00,2019-01-07 15:00:00+00:00


In [24]:
routes_weather.head()

,route_id,date,temp,wind_speed,description,precip,humidity,visibility,pressure,id,event_time
0,R-566db3fc,2019-01-21 12:00:00+00:00,28,16,Moderate or heavy snow showers,0.0,73,4,1015,245303,2024-10-10 20:13:27.063199+00:00
1,R-05a6b7e1,2019-02-13 18:00:00+00:00,16,4,Clear,0.0,63,6,1021,397270,2024-10-10 20:13:27.063199+00:00
2,R-79047854,2019-01-31 00:00:00+00:00,55,7,Sunny,0.0,57,6,1022,98159,2024-10-10 20:13:27.063199+00:00
3,R-e0263659,2019-01-22 00:00:00+00:00,75,4,Cloudy,0.0,66,6,1016,30437,2024-10-10 20:13:27.063199+00:00
4,R-dcbddbb7,2019-02-01 12:00:00+00:00,61,3,Clear,0.0,89,6,1011,259995,2024-10-10 20:13:27.063199+00:00


In [25]:
def assign_feature_group_data(feature_group_data, table_names):
    # Print available keys in feature_group_data for debugging
    print("Available keys in feature_group_data:", list(feature_group_data.keys()))
    
    # Dictionary to store dynamically assigned DataFrames
    assigned_dataframes = {}

    # Dynamically assign the DataFrame from feature_group_data to the dictionary
    for table_name in table_names:
        if table_name in feature_group_data:
            assigned_dataframes[table_name] = feature_group_data[table_name]
            print(f"Assigned {table_name} to assigned_dataframes.")
        else:
            print(f"KeyError: '{table_name}' not found in feature_group_data.")
    
    return assigned_dataframes

# List of table names matching the keys exactly as they appear in 'feature_group_data'
table_names = ['city_weather', 'routes_weather', 'traffic_table', 'truck_schedule_table', 'trucks_table']

# Call the function to assign feature group data to variables
assigned_dataframes = assign_feature_group_data(feature_group_data, table_names)

# Now you can access the DataFrames from the dictionary
print(assigned_dataframes['truck_schedule_table'].head())


Available keys in feature_group_data: ['city_weather', 'drivers_table', 'routes_table', 'routes_weather', 'traffic_table', 'truck_schedule_table', 'trucks_table']
Assigned city_weather to assigned_dataframes.
Assigned routes_weather to assigned_dataframes.
Assigned traffic_table to assigned_dataframes.
Assigned truck_schedule_table to assigned_dataframes.
Assigned trucks_table to assigned_dataframes.
   truck_id    route_id            departure_date         estimated_arrival  \
0  23244180  R-b1e8e9c8 2019-02-06 07:00:00+00:00 2019-02-07 04:09:00+00:00   
1  10234289  R-d053f5bc 2019-01-22 07:00:00+00:00 2019-01-22 22:51:00+00:00   
2  26739096  R-bf6ee666 2019-01-07 07:00:00+00:00 2019-01-07 15:54:36+00:00   
3  20067958  R-abc9e78b 2019-01-04 07:00:00+00:00 2019-01-05 05:28:12+00:00   
4  23639934  R-7dac228a 2019-01-31 07:00:00+00:00 2019-01-31 13:40:12+00:00   

   delay    id                       event_time  
0      1  7633 2024-10-10 20:13:27.142196+00:00  
1      0  4651 2024-1

In [26]:

dfs = {
    'city_weather': city_weather,
    'drivers_table': drivers_table,
    'routes_table': routes_table,
    'routes_weather': routes_weather,
    'traffic_table': traffic_table,
    'truck_schedule_table': truck_schedule_table,
    'trucks_table': trucks_table
}


def drop_columns_from_tables(dfs, columns_to_drop):
    """
    Drops specified columns from all DataFrames in the provided dictionary.
    
    Parameters:
    - dfs (dict): A dictionary where the keys are DataFrame names and the values are pandas DataFrames.
    - columns_to_drop (list): A list of column names to be dropped from each DataFrame.
    
    Returns:
    - dict: A dictionary of DataFrames with the specified columns removed.
    """
    
    # Loop through each DataFrame in the dictionary
    for table_name, df in dfs.items():
        # Check if the columns exist in the DataFrame before dropping
        cols_to_drop = [col for col in columns_to_drop if col in df.columns]
        
        # If there are columns to drop, drop them
        if cols_to_drop:
            dfs[table_name] = df.drop(columns=cols_to_drop, inplace=True)
            print(f"Dropped columns {cols_to_drop} from {table_name}.")
        else:
            print(f"No matching columns to drop in {table_name}.")
    
    return dfs

# Example usage:
columns_to_drop = ['id', 'event_time']  # Example columns that might be present in multiple tables
feature_group_data_cleaned = drop_columns_from_tables(feature_group_data, columns_to_drop)




Dropped columns ['id', 'event_time'] from city_weather.
Dropped columns ['id', 'event_time'] from drivers_table.
Dropped columns ['id', 'event_time'] from routes_table.
Dropped columns ['id', 'event_time'] from routes_weather.
Dropped columns ['id', 'event_time'] from traffic_table.
Dropped columns ['id', 'event_time'] from truck_schedule_table.
Dropped columns ['id', 'event_time'] from trucks_table.


In [27]:
def drop_duplicates_from_dfs(dfs, columns_to_check):   
    for df_name, df in dfs.items():
        if df_name in columns_to_check:
            subset_columns = columns_to_check[df_name]
            
            # Check if all subset columns exist in the DataFrame
            missing_columns = [col for col in subset_columns if col not in df.columns]
            if missing_columns:
                print(f"Skipping {df_name}: Missing columns {missing_columns}")
                continue

            before_dropping = df.shape[0]  # Count the number of rows before dropping duplicates
            df.drop_duplicates(subset=subset_columns, inplace=True)
            after_dropping = df.shape[0]  # Count the number of rows after dropping duplicates
            print(f"Dropped {before_dropping - after_dropping} duplicate rows from {df_name} based on columns: {subset_columns}")
        else:
            print(f"No duplicate check columns provided for {df_name}")

# Dictionary where the key is the DataFrame name, and the value is the subset of columns to check for duplicates
columns_to_check = {
    'city_weather': ['city_id', 'datetime'],
    'routes_weather': ['route_id', 'date'],
    'trucks_table': ['truck_id'],
    'drivers_table': ['driver_id'],
    'routes_table': ['route_id', 'destination_id', 'origin_id'],
    'truck_schedule_table': ['truck_id', 'route_id', 'departure_date']
}

# Call the function to drop duplicates from the DataFrames
drop_duplicates_from_dfs(dfs, columns_to_check)

# Check the result by printing the modified DataFrames
for df_name, df in dfs.items():
    print(f"\n{df_name} DataFrame after dropping duplicates:")
    print(df)


Dropped 707 duplicate rows from city_weather based on columns: ['city_id', 'datetime']
Dropped 0 duplicate rows from drivers_table based on columns: ['driver_id']
Dropped 0 duplicate rows from routes_table based on columns: ['route_id', 'destination_id', 'origin_id']
Dropped 0 duplicate rows from routes_weather based on columns: ['route_id', 'date']
No duplicate check columns provided for traffic_table
Dropped 0 duplicate rows from truck_schedule_table based on columns: ['truck_id', 'route_id', 'departure_date']
Dropped 0 duplicate rows from trucks_table based on columns: ['truck_id']

city_weather DataFrame after dropping duplicates:
          city_id  temp  wind_speed           description  precip  humidity  \
0      C-731988ba    52           1  Patchy rain possible     0.0        80   
1      C-841ebdcb    66           6         Partly cloudy     0.0        84   
2      C-ef47bdcd    10           6                 Sunny     0.0        57   
3      C-ff8c0c3c    34           8      

In [29]:
def drop_duplicates_from_dfs(dfs, columns_to_check):   
    for df_name, df in dfs.items():
        if df_name in columns_to_check:
            subset_columns = columns_to_check[df_name]
            
            # Check if all subset columns exist in the DataFrame
            missing_columns = [col for col in subset_columns if col not in df.columns]
            if missing_columns:
                print(f"Skipping {df_name}: Missing columns {missing_columns}")
                continue

            before_dropping = df.shape[0]  # Count the number of rows before dropping duplicates
            df.drop_duplicates(subset=subset_columns, inplace=True)
            after_dropping = df.shape[0]  # Count the number of rows after dropping duplicates
            print(f"Dropped {before_dropping - after_dropping} duplicate rows from {df_name} based on columns: {subset_columns}")
            
            # Print the number of rows and columns in each DataFrame after dropping duplicates
            print(f"{df_name} DataFrame after dropping duplicates: {df.shape[0]} rows, {df.shape[1]} columns")
        else:
            print(f"No duplicate check columns provided for {df_name}")

# Dictionary where the key is the DataFrame name, and the value is the subset of columns to check for duplicates
columns_to_check = {
    'city_weather': ['city_id', 'datetime'],
    'routes_weather': ['route_id', 'date'],
    'trucks_table': ['truck_id'],
    'drivers_table': ['driver_id'],
    'routes_table': ['route_id', 'destination_id', 'origin_id'],
    'truck_schedule_table': ['truck_id', 'route_id', 'departure_date']
}

# Call the function to drop duplicates from the DataFrames
drop_duplicates_from_dfs(dfs, columns_to_check)

# Check the result by printing the modified DataFrames' shapes
for df_name, df in dfs.items():
    print(f"\n{df_name} DataFrame has {df.shape[0]} rows and {df.shape[1]} columns after dropping duplicates.")


Dropped 0 duplicate rows from city_weather based on columns: ['city_id', 'datetime']
city_weather DataFrame after dropping duplicates: 38228 rows, 9 columns
Dropped 0 duplicate rows from drivers_table based on columns: ['driver_id']
drivers_table DataFrame after dropping duplicates: 1100 rows, 9 columns
Dropped 0 duplicate rows from routes_table based on columns: ['route_id', 'destination_id', 'origin_id']
routes_table DataFrame after dropping duplicates: 1896 rows, 5 columns
Dropped 0 duplicate rows from routes_weather based on columns: ['route_id', 'date']
routes_weather DataFrame after dropping duplicates: 396149 rows, 9 columns
No duplicate check columns provided for traffic_table
Dropped 0 duplicate rows from truck_schedule_table based on columns: ['truck_id', 'route_id', 'departure_date']
truck_schedule_table DataFrame after dropping duplicates: 10292 rows, 5 columns
Dropped 0 duplicate rows from trucks_table based on columns: ['truck_id']
trucks_table DataFrame after dropping du

FEATURE ENGINEERING

Merge Route_weather with Schedule data * create new data frame from scheduledf and add estimated_arrival and departure_date columns estimated_arriaval= df[estimated_arrival].dt.ceil("6H') departure_date= df[departure_date].dt.floor("6H")

In [32]:
truck_schedule_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10292 entries, 0 to 10291
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   truck_id           10292 non-null  int64              
 1   route_id           10292 non-null  object             
 2   departure_date     10292 non-null  datetime64[us, UTC]
 3   estimated_arrival  10292 non-null  datetime64[us, UTC]
 4   delay              10292 non-null  int64              
dtypes: datetime64[us, UTC](2), int64(2), object(1)
memory usage: 402.2+ KB


In [33]:
routes_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396149 entries, 0 to 396148
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   route_id     396149 non-null  object             
 1   date         396149 non-null  datetime64[us, UTC]
 2   temp         396149 non-null  int64              
 3   wind_speed   396149 non-null  int64              
 4   description  396149 non-null  object             
 5   precip       396149 non-null  float64            
 6   humidity     396149 non-null  int64              
 7   visibility   396149 non-null  int64              
 8   pressure     396149 non-null  int64              
dtypes: datetime64[us, UTC](1), float64(1), int64(5), object(2)
memory usage: 27.2+ MB


In [34]:
import pandas as pd

def process_dates(df, estimated_arrival_column, departure_date_column):
    # Function to convert to UTC if not already timezone-aware
    def convert_to_utc(series):
        if series.dt.tz is None:  # Check if the series is not timezone-aware
            return series.dt.tz_localize('UTC')
        else:
            return series.dt.tz_convert('UTC')  # Convert to UTC if it already has a timezone

    # Step 1: Create a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()

    # Step 2: Convert both columns to datetime format and set to UTC timezone
    df_copy[estimated_arrival_column] = convert_to_utc(pd.to_datetime(df_copy[estimated_arrival_column], errors='coerce'))
    df_copy[departure_date_column] = convert_to_utc(pd.to_datetime(df_copy[departure_date_column], errors='coerce'))

    # Step 3: Apply ceil operation to estimated arrival
    df_copy[estimated_arrival_column] = df_copy[estimated_arrival_column].dt.ceil("6H")

    # Step 4: Floor the 'departure_date' column to the nearest 6 hours
    df_copy[departure_date_column] = df_copy[departure_date_column].dt.floor("6H")

    # Step 5: Create a new column 'date' with date ranges between 'departure_date' and 'estimated_arrival'
    df_copy['date'] = [
        pd.date_range(start=row[departure_date_column], end=row[estimated_arrival_column], freq='6H')
        for index, row in df_copy.iterrows()
    ]
    
    # Step 6: Explode the 'date' column to separate rows for each date in the range
    df_copy = df_copy.explode('date').reset_index(drop=True)

    return df_copy


# Define target table(s) to process
target_tables = ['truck_schedule_table']  # Your actual table names from the 'dfs' dictionary
departure_date_column = 'departure_date'
estimated_arrival_column = 'estimated_arrival'

# Process the DataFrames
for table_name in list(dfs.keys()):  # Use list(dfs.keys()) to avoid changing the dictionary during iteration
    df = dfs[table_name]
    if table_name in target_tables:
        # Create a copy and process dates
        result_df_copy = process_dates(df, estimated_arrival_column, departure_date_column)
        
        # Merge with routes_weather on 'route_id' and 'date'
        scheduled_weather = pd.merge(result_df_copy, dfs['routes_weather'], on=['route_id', 'date'], how='left')

        # Display or log the merged DataFrame for review
        print(f"Processed {table_name} with route weather data merge:\n", scheduled_weather.head())

        # Save the processed DataFrame under a new key in the 'dfs' dictionary
        dfs[f'{table_name}_processed'] = scheduled_weather


Processed truck_schedule_table with route weather data merge:
    truck_id    route_id            departure_date         estimated_arrival  \
0  23244180  R-b1e8e9c8 2019-02-06 06:00:00+00:00 2019-02-07 06:00:00+00:00   
1  23244180  R-b1e8e9c8 2019-02-06 06:00:00+00:00 2019-02-07 06:00:00+00:00   
2  23244180  R-b1e8e9c8 2019-02-06 06:00:00+00:00 2019-02-07 06:00:00+00:00   
3  23244180  R-b1e8e9c8 2019-02-06 06:00:00+00:00 2019-02-07 06:00:00+00:00   
4  23244180  R-b1e8e9c8 2019-02-06 06:00:00+00:00 2019-02-07 06:00:00+00:00   

   delay                      date  temp  wind_speed        description  \
0      1 2019-02-06 06:00:00+00:00  52.0         7.0              Clear   
1      1 2019-02-06 12:00:00+00:00  37.0         4.0  Patchy light rain   
2      1 2019-02-06 18:00:00+00:00  36.0         4.0  Patchy light rain   
3      1 2019-02-07 00:00:00+00:00  52.0         7.0              Clear   
4      1 2019-02-07 06:00:00+00:00  52.0         7.0              Clear   

   precip  

Merge the resultant dataframe with route_weather on route_id and date (left)

In [35]:
scheduled_weather

,truck_id,route_id,departure_date,estimated_arrival,delay,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,23244180,R-b1e8e9c8,2019-02-06 06:00:00+00:00,2019-02-07 06:00:00+00:00,1,2019-02-06 06:00:00+00:00,52.0,7.0,Clear,0.0,86.0,6.0,1020.0
1,23244180,R-b1e8e9c8,2019-02-06 06:00:00+00:00,2019-02-07 06:00:00+00:00,1,2019-02-06 12:00:00+00:00,37.0,4.0,Patchy light rain,0.0,99.0,6.0,1010.0
2,23244180,R-b1e8e9c8,2019-02-06 06:00:00+00:00,2019-02-07 06:00:00+00:00,1,2019-02-06 18:00:00+00:00,36.0,4.0,Patchy light rain,0.0,98.0,6.0,1010.0
3,23244180,R-b1e8e9c8,2019-02-06 06:00:00+00:00,2019-02-07 06:00:00+00:00,1,2019-02-07 00:00:00+00:00,52.0,7.0,Clear,0.0,85.0,6.0,1021.0
4,23244180,R-b1e8e9c8,2019-02-06 06:00:00+00:00,2019-02-07 06:00:00+00:00,1,2019-02-07 06:00:00+00:00,52.0,7.0,Clear,0.0,85.0,6.0,1021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59674,48204624,R-f5d9a32d,2019-01-15 06:00:00+00:00,2019-01-22 06:00:00+00:00,0,2019-01-22 06:00:00+00:00,30.0,14.0,Moderate snow,0.0,83.0,3.0,1011.0
59675,10787409,R-ef208446,2019-01-13 06:00:00+00:00,2019-01-14 00:00:00+00:00,0,2019-01-13 06:00:00+00:00,68.0,1.0,Clear,0.0,82.0,6.0,1017.0
59676,10787409,R-ef208446,2019-01-13 06:00:00+00:00,2019-01-14 00:00:00+00:00,0,2019-01-13 12:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59677,10787409,R-ef208446,2019-01-13 06:00:00+00:00,2019-01-14 00:00:00+00:00,0,2019-01-13 18:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
